In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm #시간 측정용

# ---------- 한글 --------- #
import MeCab
mec = MeCab.Tagger()

pd.set_option("display.max_rows", 500)

In [2]:
# ----------------------- 정규표현식 ----------------------- #
import re
is_ProductName = re.compile("[a-zA-Z0-9]+")
# is_KoScalar = re.compile("[0-9]+[단칸롤개팩장캔종입원매분시일초포호쿼마단부인박]{1}[터리계분용스]{0,1}")

In [3]:
def MCSplit(keyword):
    ret = []
    for i in keyword.split("\n"):
        app = i.split("\t")[0]
        if app != 'EOS':
            ret.append(app)
        else:
            return ret
    return ret
def subSChar(keyword):
    ret = re.sub("[-=+,.!@#$%^&*(\)[\]<\>{\}\'\"/_|ㄱ-ㅎ|ㅏ-ㅣ]"," ",keyword)
    return ret
def KoEnMorphs(keyword):
    x = (subSChar(keyword.lower())).split()
    for i in range(0,len(x)):
        for j in is_ProductName.findall(x[i]):
            x[i] = x[i].replace(j," " + j + " ")
    x = (" ".join(x)).split()
    ret = []
    for i in x:
        if is_ProductName.fullmatch(i) == None:
            ret += MCSplit(mec.parse((i)))
        else:
            ret.append(i)
    return ret

In [4]:
beh = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv")
tra = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv")
cus = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-03.고객 Demographic 정보.csv")
cla = pd.read_csv("./Data/제6회 L.POINT Big Data Competition-분석용데이터-04.상품분류 정보.csv")

E:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# ------------------------------- 대분류에 키워드 담기 ------------------------------- #

In [6]:
kwd_Clac_nm1 = np.load("./Keyword in Clac_nm123/대분류 pd_c 키워드 파일.npy")
kwd_Clac_nm2 = np.load("./Keyword in Clac_nm123/중분류 pd_c 키워드 파일.npy")
kwd_Clac_nm3 = np.load("./Keyword in Clac_nm123/소분류 pd_c 키워드 파일.npy")

In [7]:
clac_nm1_Name = np.load("./Keyword in Clac_nm123/대분류 pd_c Name 파일.npy")
clac_nm2_Name = np.load("./Keyword in Clac_nm123/중분류 pd_c Name 파일.npy")
clac_nm3_Name = np.load("./Keyword in Clac_nm123/소분류 pd_c Name 파일.npy")

In [11]:
# 카테고리별 토큰 생성
from keras.preprocessing.text import Tokenizer

num_words = 50000
 
# 대분류 Tokenizer
Tokenizer_Clac1 = []
for i in tqdm(range(0,len(kwd_Clac_nm1))):
    Tokenizer_Clac1.append(Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True))
    Tokenizer_Clac1[i].fit_on_texts(kwd_Clac_nm1[i])
# 대분류 Tokenizer
Tokenizer_Clac2 = []
for i in tqdm(range(0,len(kwd_Clac_nm2))):
    Tokenizer_Clac2.append(Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True))
    Tokenizer_Clac2[i].fit_on_texts(kwd_Clac_nm2[i])

100%|████████████████████████████████████████████████████████████████████████████████| 332/332 [00:08<00:00, 41.18it/s]


In [12]:
# 토큰 저장
import pickle
import os

try:
    if not(os.path.isdir('./Tokenizer/Clac_nm')):
        os.makedirs(os.path.join('./Tokenizer/Clac_nm'))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("디렉토리 생성 실패.")

# Save Tokenizer 대분류 , 중분류
for i in range(0,len(Tokenizer_Clac1)):
    with open('./Tokenizer/Clac_nm/Tokenizer_Clac1_{0}.pickle'.format(i), 'wb') as handle:
        pickle.dump(Tokenizer_Clac1[i], handle, protocol=pickle.HIGHEST_PROTOCOL)
for i in range(0,len(Tokenizer_Clac2)):
    with open('./Tokenizer/Clac_nm/Tokenizer_Clac2_{0}.pickle'.format(i), 'wb') as handle:
        pickle.dump(Tokenizer_Clac2[i], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
# 계산식
def SequenceCalc(sequence,tok):
    ret = 0
    for i in sequence:
        if len(i) != 0:
            # 순위가 높을수록 해당 카테고리일 확률이 높음
            # 1/순위 * (단어가 나온 횟수 / 해당 문서의 총 길이)
            ret += (1/i[0])*(tok.word_counts[tok.index_word[i[0]]]/tok.document_count)
        else:
            ret += 0.0
    return ret
# 대분류 출력
def Kwd2Category1(keyword):
    temp = []
    # 순위 계산
    for tok in Tokenizer_Clac1:
        temp.append(SequenceCalc(tok.texts_to_sequences(KoEnMorphs(keyword)),tok))
    # 정렬해준다.
    tempSrt = sorted(temp, key = lambda x:float(x))
    
    index = []
    for i in reversed(range(0,len(tempSrt))):
        if tempSrt[i] != 0:
            index.append(temp.index(tempSrt[i]))
    # 인덱스를 찾아준다.
    ret = []
    for i in index:
        ret.append(clac_nm1_Name[i])
    return ret[:5]

# 중분류 출력 -> 
def Kwd2Category2(keyword):
    temp = []
    # 순위 계산
    for tok in Tokenizer_Clac2:
        temp.append(SequenceCalc(tok.texts_to_sequences(KoEnMorphs(keyword)),tok))
    # 정렬해준다.
    tempSrt = sorted(temp, key = lambda x:float(x))
    
    index = []
    for i in reversed(range(0,len(tempSrt))):
        if tempSrt[i] != 0:
            index.append(temp.index(tempSrt[i]))
    # 인덱스를 찾아준다.
    ret = []
    for i in index:
        ret.append(clac_nm2_Name[i])
    return ret[:5]